# Working with AprilTags

This notebook will show you how to work with AprilTags for visualization and pose estimation. See the Apriltags Markdown file and Fisheye Calibration Markdown file for the theory behind it.

#### Imports and Global Parameters

In [1]:
import numpy as np
from ATDetector import ATDetector
import cv2
from time import sleep
from IPython.display import clear_output
import matplotlib.pyplot as plt
%matplotlib inline

#Scale Factor for Showing Images
SCALE = .3

####  Camera
To use the AprilTags, we need information about the camera we are using, specifically:
- Camera Resolution
- Intrinsic matrix for pose estimation
- Distortion Array for fisheye Undistortion
- Fisheye flag if camera is fisheye

In [2]:
camera = {}
#Camera Resolution
camera["res"] = (1280, 760)
#Camera Intrinsic Matrix (3x3)
camera["K"] = np.array(
    [[631.6058624841243, 0.0, 673.9002987027918], [0.0, 627.4303222760955, 380.85431690312384], [0.0, 0.0, 1.0]])
#The non-default elements of the K array, in the AprilTag specification
camera["params"] = [631.605, 627.43, 673.9, 380.85]
#Fisheye Camera Distortion Matrix
camera["D"] = np.array(
    [[-0.031080677599846774], [-0.006061559072085696], [-0.0011641369792770276], [0.00028577486827623653]])
#Fisheye flag
camera["fisheye"] = True


#### Tag Information
To use the AprilTags, we need information about the tags we are using, specifically:
- Tag Families
- Size of the tag (in meters)
- (Optional) Location of the tags in the world frame

In [3]:
tags = {}
#To add more families, seperate with a space instead of using a list
families = "tagStandard52h13"
#Size of Tag in Meters
tag_size = .04
#Tag Locations
tag_locations = {}
tag_locations[4] = np.array([[.127],[0.],[0.]])
tag_locations[3] = np.array([[-.127],[0.],[0.]])

In [4]:
#Create Detector
detect=ATDetector(families,tag_size,camera)

### Working with Images

There are 3 main functions to use when working with a single image:
- Visualize Image Detections -> Takes a filename and displays information about the tags overlaid on the image.
- Estimate Image Position -> Takes a filename and prints the estimated camera location according to each detected tag.
- Estimate Image Orientation -> Takes a filename and prints the estimated camera orientation according to each detected tag.

In [5]:
#Example Image
fname = "Images/two_tags_pose.png"
#Overlay detections on the Image
detect.visualize_image_detections(fname)

In [6]:
detect.estimate_image_position(fname,tag_locations)

Tag 3 estimates the camera at:
X:0.08358853637084757
Y:0.09919716272553836
Z:0.9832652919425806

Tag 4 estimates the camera at:
X:0.0851904804692395
Y:0.11017903407917169
Z:0.9958769917583619



In [7]:
detect.estimate_image_orientation(fname)

Tag 3 estimates the camera pose with:
Theta_X:1.6655647960450366
Theta_Y:4.895864583564171
Theta_Z:179.53814273891854
Tag 4 estimates the camera pose with:
Theta_X:1.1738192724042376
Theta_Y:6.062555705573457
Theta_Z:179.76645915191907


### Working with Videos

There are 3 main functions to use when working with a single image:
- Visualize Video Detections -> Takes a filename and displays information about the tags overlaid on the video.<br>
- Estimate Video Position -> Takes a filename and returns a matrix with the camera position for each frame. <br>
- Estimate Video Orientation -> Takes a filename and returns a matrix with the camera orientation for each frame. <br>


In [5]:
#Overlay detections on the Video
fname="Videos/x_and_z_2.h264"
detect.visualize_video_detections(fname)

In [ ]:
#Get Positions for each frame
positions = detect.estimate_video_position(fname,tag_locations)
#Get Orientations for each frame
orientations = detect.estimate_video_orientation(fname)

### Visualizing Position Estimation

The cell below plots a graph with the position of the camera in the previous video. As you can see it is not very accurate and very noisy. Stay tuned for further improvements to make this better with Kalman filtering!


In [ ]:
plt.figure()
#Create Color Wheel
phi = np.linspace(0, 2*np.pi, positions.shape[1])
x = np.sin(phi)
y = np.cos(phi)
rgb_cycle = np.vstack((           
    .5*(1.+np.cos(phi)), 
    .5*(1.+np.cos(phi+2*np.pi/3)), 
    .5*(1.+np.cos(phi-2*np.pi/3)))).T 
#Animation Loop
for i in range(positions.shape[1]):
    plt.scatter(positions[0,:i],positions[2,:i],c=rgb_cycle[:i])
    plt.show()
    sleep(0.1)
    clear_output(wait=True)